In [94]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sqlalchemy import create_engine

In [95]:
conn = sqlite3.connect('database.sqlite')

In [96]:
df = pd.read_sql("""
                        SELECT *
                        from loan
                        WHERE grade = 'A';
                        
                       """, con=conn)

## Extract Base Data - Loan Data

In [158]:
df_base = df[['issue_d','addr_state','annual_inc','annual_inc_joint','dti','grade','tot_cur_bal','home_ownership']].copy()
clean_zip = df['zip_code'].str.rstrip('xx')
df_base['zip_code_3digit']= clean_zip

df_base.head(20)

,issue_d,addr_state,annual_inc,annual_inc_joint,dti,grade,tot_cur_bal,home_ownership,zip_code_3digit
0,Dec-2018,UT,65000,,23.6,A,249643,MORTGAGE,840
1,Dec-2018,TX,40000,,12.87,A,66069,MORTGAGE,750
2,Dec-2018,CT,30000,85000,43.32,A,210356,MORTGAGE,062
3,Dec-2018,PA,100000,,16.42,A,192652,MORTGAGE,154
4,Dec-2018,CT,35000,,32.65,A,53018,MORTGAGE,068
5,Dec-2018,CA,161000,,10.99,A,370187,MORTGAGE,933
6,Dec-2018,NV,130000,239000,9.29,A,422806,MORTGAGE,895
7,Dec-2018,WA,78000,,8.58,A,289602,MORTGAGE,980
8,Dec-2018,WA,71250,,5.71,A,7510,RENT,984
9,Dec-2018,NY,40000,,12.81,A,28062,RENT,100


## Establish Connection to MySQL Database

In [159]:
connection_string = 'root:MyPassword@127.0.0.1:3306/loan_db'
engine = create_engine('mysql+pymysql://{}'.format(connection_string))
engine.table_names()

['home_value_zip', 'income_data', 'income_zip', 'loan_data', 'state_&_zip']

### Load Loan Data to MySQL

In [160]:
df_base.to_sql(name='loan_data',con=engine, if_exists='replace')

## Second Dataset - Home Values

In [161]:
zip_path = './zecon/Zip_time_series.csv'
zip_code_table = pd.read_csv(zip_path)

zip_home_val = zip_code_table[['Date','RegionName','MedianListingPrice_AllHomes','MedianRentalPrice_AllHomes']].copy()

zip_home_val2 = zip_home_val.rename(columns={'RegionName':'Zip_Code'})
zip_home_val2.head(3)

,Date,Zip_Code,MedianListingPrice_AllHomes,MedianRentalPrice_AllHomes
0,1996-04-30,1001,NaN,NaN
1,1996-04-30,1002,NaN,NaN
2,1996-04-30,1005,NaN,NaN


### Group by Zip Code & Load to MySQL

In [122]:
Avg_Home_Val_ZipCode = zip_home_val2.groupby('Zip_Code').median()
Avg_Home_Val_ZipCode.to_sql(name='home_value_zip',con=engine, if_exists='replace')

## Third Dataset - Median Household Income

In [162]:
income_path = './income/kaggle_income.csv'
income_table = pd.read_csv(income_path,encoding = "ISO-8859-1")

income = income_table[['State_Name','State_ab','City','Zip_Code','Mean']].copy()
income_table.head()

,id,State_Code,State_Name,State_ab,County,City,Place,Type,Primary,Zip_Code,Area_Code,ALand,AWater,Lat,Lon,Mean,Median,Stdev,sum_w
0,1011000,1,Alabama,AL,Mobile County,Chickasaw,Chickasaw city,City,place,36611,251,10894952,909156,30.771450,-88.079697,38773,30506,33101,1638.260513
1,1011010,1,Alabama,AL,Barbour County,Louisville,Clio city,City,place,36048,334,26070325,23254,31.708516,-85.611039,37725,19528,43789,258.017685
2,1011020,1,Alabama,AL,Shelby County,Columbiana,Columbiana city,City,place,35051,205,44835274,261034,33.191452,-86.615618,54606,31930,57348,926.031000
3,1011030,1,Alabama,AL,Mobile County,Satsuma,Creola city,City,place,36572,251,36878729,2374530,30.874343,-88.009442,63919,52814,47707,378.114619
4,1011040,1,Alabama,AL,Mobile County,Dauphin Island,Dauphin Island,Town,place,36528,251,16204185,413605152,30.250913,-88.171268,77948,67225,54270,282.320328


### Create Mean Household Income Table

In [163]:
income_zip = income.groupby('Zip_Code').agg({'Mean':'mean'})
income_zip.reset_index()
income_zip2 = income_zip.rename(columns={'Mean':'Mean_Income'})
income_zip2.head()

,Mean_Income
Zip_Code,
601,22496.333333
602,15078.400000
603,26046.800000
605,27497.500000
606,19194.000000


### Create State & Zip Code Table

In [148]:
State_Zip = income_table[['State_Name','State_ab','Zip_Code']].copy()
State_Zip.head()

,State_Name,State_ab,Zip_Code
0,Alabama,AL,36611
1,Alabama,AL,36048
2,Alabama,AL,35051
3,Alabama,AL,36572
4,Alabama,AL,36528


### Load Household Income & State/Zip Tables to MySQL DB

In [150]:
income_zip2.to_sql(name='income_zip',con=engine, if_exists='replace')
State_Zip.to_sql(name='state_&_zip',con=engine, if_exists='replace')